In [1]:
%pip install requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import gradio as gr

In [ ]:
pip install --upgrade jupyter ipywidgets


In [ ]:
import gradio as gr

def hello_world(image):
    # 이미지 입력을 받았으므로 이를 그대로 반환하거나 다른 처리를 할 수 있습니다.
    return image

demo = gr.Interface(fn=hello_world, inputs="image", outputs="image")

demo.launch()

In [2]:
import gradio as gr
import random
import requests

def request_chatgpt(input_text, histories, system_text):
    endpoint = "https://st004-oa.openai.azure.com/"  
    api_key = "f7379e4b1f3346de8f5a55d5305a360f"   
    deployment_name = "gpt-4o"
    ENDPOINT_URL = f"{endpoint}openai/deployments/{deployment_name}/chat/completions?api-version=2024-02-15-preview"

    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }

    body = {
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    messages = []

    messages.append({
        "role": "system",
        "content": {
            "type": "text",
            "text": system_text
        }
    })

    for history in histories:
        messages.append({
            "role": "assistant",
            "content": {
                "type": "text",
                "text": history[0]
            }
        })
        messages.append({
            "role": "user",
            "content": {
                "type": "text",
                "text": history[1]
            }
        })

    user_message = {
        "role": "user",
        "content": {
            "type": "text",
            "text": input_text
        }
    }
    messages.append(user_message)

    body.update({"messages": messages})

    response = requests.post(ENDPOINT_URL, headers=headers, json=body)
    response_json = response.json()

    if response.status_code == 200:
        response_text = response_json['choices'][0]['message']['content']
        print(response_text)
        return response_text
    else:
        print(response.text)
        return None

def click_send_button(user_message, chatbot_histories, system_message):
    response_message = request_chatgpt(user_message, chatbot_histories, system_message)

    if response_message is None:
        return "", chatbot_histories

    print(chatbot_histories)
    chatbot_histories.append((user_message, response_message))
    return "", chatbot_histories

with gr.Blocks() as demo:
    with gr.Column():
        chatbot = gr.Chatbot(label="GPT")
        system_textbox = gr.Textbox(label="시스템메시지")

    with gr.Row():
        input_textbox = gr.Textbox(label="입력", scale=5)
        send_button = gr.Button("전송")

    send_button.click(fn=click_send_button, inputs=[input_textbox, chatbot, system_textbox], outputs=[input_textbox, chatbot])
    input_textbox.submit(click_send_button, inputs=[input_textbox, chatbot, system_textbox], outputs=[input_textbox, chatbot])

# Gradio 인터페이스 실행
demo.launch()
